In [39]:
from scipy.optimize import minimize
import numpy as np

In [52]:
# error function parameters  modes
eta = 9.14709251
'''rho = pm.Uniform("rho", lower=0, upper=10)
beta_L = pm.Uniform("beta_L", lower=0, upper=3)
beta_H = pm.Uniform("beta_H", lower=0, upper=6)'''
rho = 0.23346471
beta_L = 1.18626015
beta_H = 0.66452116
# rho_2 = pm.Uniform("rho_2", lower=0, upper=1)
beta_H2 = 1.9791696
# alpha_1 = pm.Normal("alpha_1", mu=10, sigma=5)
alpha_L = 9.88180657
alpha_1 = 9.04729438
alpha_2 = 20.49538774

sigma_lg_Phi = 0.09441348   # standard deviation of the log of the error function

# computational cost
C_L = .994 + 1.845  # C_L1 + C_L2 in CPU hours
C_H = 178.813

q = C_H / C_L

# objective error
err_0 = .01 # goal error


In [67]:
750*(.994 + 1.845)*1200/56

45626.78571428572

In [53]:
def err_func(n_LF, n_HF):
    Phi = eta * (rho / ( (n_LF + n_HF) * (n_LF + alpha_L)**(beta_L-1) * (n_HF + alpha_1)**(beta_H-1) ) + 1 / (alpha_2 + n_HF)**(beta_H2))
    # median_Phi = 10**lg_Phi
    return Phi

def err_mean(n_LF, n_HF):
    gamma = np.exp(1/2 * (sigma_lg_Phi * np.log(10))**2)
    return gamma * err_func(n_LF, n_HF)

def comp_bud(n_L, n_H):
    return C_L * n_L + C_H * n_H

In [54]:


# # Define the objective function
def objective(vars):
    n_L, n_H = vars
    return comp_bud(n_L, n_H)

# Define the constraint function
def constraint(vars):
    n_L, n_H = vars
    return err_mean(n_L,n_H) - err_0

# Initial guess for the variables
initial_guess = [500, 20]

# Define the constraint dictionary
con = {'type': 'eq', 'fun': constraint}

# Perform the optimization
result = minimize(objective, initial_guess, constraints=[con])

# Extract the optimal values of x and y
x_opt, y_opt = result.x

print(f"Optimal values: x = {x_opt}, y = {y_opt}")
print(f"Function value at optimal points: f(x, y) = {objective((x_opt, y_opt))}")

Optimal values: x = 644.7931515609873, y = 17.38780996305297
Function value at optimal points: f(x, y) = 4939.734220205033


In [55]:
comp_bud(x_opt, y_opt) / 56

88.20953964651845

In [56]:
comp_bud(x_opt, y_opt) / 56 * 1200 

105851.44757582214

In [64]:
comp_bud(750, 10) / 56 * 1200 

83943.85714285714

In [57]:
n_LF = 564
n_HF = 21

In [58]:
err_mean(n_LF,n_HF)

0.009463563227507668

In [59]:
comp_bud(n_LF, n_HF) / 56 * 1200

114777.19285714286

In [60]:
err_mean(0, 66.3)

0.09312817422204356

In [61]:
comp_bud(0, 66.3) / 56 * 1200

254042.18357142853

In [62]:
err_func(10000, 0)

0.023269996046344138

In [63]:
# error function at integer n_LF and n_HF
n_LF = 78
n_HF = 9
phi = eta * (rho * n_LF**(-beta_L) + n_HF**(-beta_H))
print("phi at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), "%.2f" % (phi * 100) + "%")

# computational cost at integer n_LF and n_HF
cpus = 56 # frontera
C = C_L * n_LF + C_H * n_HF
print("C at n_LF = %d, n_HF = %d:" % (n_LF, n_HF), C, "CPU hours", "= %.2f" % (C/cpus), "node hours")

phi at n_LF = 78, n_HF = 9: 213.62%
C at n_LF = 78, n_HF = 9: 1830.759 CPU hours = 32.69 node hours
